<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT21/blob/jacwah/Lab_3/jacwah_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 3: Iterative methods**
**Jacob Wahlgren**

# **Abstract**

Iterative methods is a way to find approximate solutions to equations. Implementations of two algorithms for systems of linear equations (Jacobi, Gauss-Seidel), and one algorithm for scalar nonlinear equations (Newton) are presented. Randomized test cases verify the correctness of the implementations.

#**About the code**

The code was written by the author (Jacob Wahlgren), based on a template by Johan Hoffman.

In [1]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2020 Johan Hoffman (jhoffman@kth.se)
# Copyright (C) 2021 Jacob Wahlgren (jacobwah@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

In [2]:
# Load neccessary modules.
from google.colab import files

import numpy as np
from numpy.polynomial import Polynomial as P
import unittest
from functools import reduce

# **Introduction**

Last week focused on direct methods of solving equations. This week the focus is iterative methods, i.e. where each step of the algorithm yields a better approximation of the exact solution. For linear equations, the benefit of iterative methods is that sparse matrices can be handled faster and with less memory than for direct methods. Iterative methods are also useful to solve nonlinear equations, since they cannot be solved with the direct methods shown previously.

Jacobi iteration and Gauss-Seidel iteration are two similar methods of solving linear systems of equations $Ax=b$ based on stationary iteration. Statitionary iteration follows the pattern

$\displaystyle x^{(k+1)} = M x^{(k)} + c$

for some constant matrix $M$ and vector $c$.

Newton's method solves nonlinear equations by following the tangent line at each point.

# **Method**

Both Jacobi iteration and Gauss-Seidel iteration are based on matrix splitting, where the matrix $A$ is split into $A=A_1+A_2$. The matrix $A_1$ is chosen to be easy to invert. They are defined in section 7.7 of the lecture notes. For Gauss-Seidel $A_1=\text{diag}(A)$. Then use the following stationary system.

$\displaystyle M=I - A_1^{-1} A\\
c = A_1^{-1} b$

The iteration converges if $\|M\| < 1$. In the random test cases, this condition is tested, and if it does not hold a new instance is sampled. Sampling is slower the more dimensions are used, therefore the tests only go up to 4 dimensions.

In [3]:
def diag(A):
  D = np.copy(A)
  n = len(A)
  for i in range(n):
    for j in range(n):
      if i != j:
        D[i,j] = 0
  return D

def jacobi_iteration(A, b):
  D = diag(A)
  n = len(A)
  Di = np.linalg.inv(D)
  M = np.identity(n) - Di@A
  if np.linalg.norm(M, 2) >= 1:
    raise ValueError("Matrix A does not fulfill convergence criterion")
  c = Di@b
  x = np.zeros(n)
  while np.linalg.norm(A@x - b) > 0.00000001:
    x = M@x + c
  return x

class JacobiTest(unittest.TestCase):
  def test_random(self):
    for n in range(1, 5):
      valid = False
      while not valid:
        try:
          A = np.random.rand(n,n)
          y = np.random.rand(n)
          b = A@y
          x = jacobi_iteration(A, b)
          valid = True
        except ValueError:
          pass
      with self.subTest(n=n, A=A, y=y, b=b):
        np.testing.assert_almost_equal(A@x, b)
        np.testing.assert_almost_equal(x, y)

Since the only difference to Jacobi iteration is how to compute $A_1$, the implementation of Gauss-Seidel iteration is largely the same. For Gauss-Seidel $A_1$ is a lower triangular matrix, where the non-zero elements are equal to those of $A$.

In [4]:
def lower_triangular(A):
  L = np.copy(A)
  n = len(A)
  for i in range(n):
    for j in range(i):
      L[i,j] = 0
  return L

def gauss_seidel_iteration(A, b):
  L = lower_triangular(A)
  Li = np.linalg.inv(L)
  n = len(A)
  M = np.identity(n) - Li@A
  if np.linalg.norm(M, 2) >= 1:
    raise ValueError("Matrix A does not fulfill convergence criterion")
  c = Li@b
  x = np.zeros(n)
  while np.linalg.norm(A@x - b) > 0.00000001:
    x = M@x + c
  return x

class GaussSeidelTest(unittest.TestCase):
  def test_random(self):
    for n in range(1, 5):
      valid = False
      while not valid:
        try:
          A = np.random.rand(n,n)
          y = np.random.rand(n)
          b = A@y
          x = gauss_seidel_iteration(A, b)
          valid = True
        except ValueError:
          pass
      with self.subTest(n=n, A=A, y=y, b=b):
        np.testing.assert_almost_equal(A@x, b)
        np.testing.assert_almost_equal(x, y)

The formula for Newton's method is

$\displaystyle x^{(k+1)} = x^{(k)} - \frac{f(x^{(k)})}{f'(x^{(k)})}$

and is defined in section 8.3 of the lecture notes. The iteration is intialized by a "guess" $x_0$. The closer to the solution $x_0$ is, the faster the algorithm converges. I chose to use $x_0=0$ since the assignment specified that there should be no extra parameter.

The implementation of Newton's method is based on the pseudocode in Algorithm 8.2 in the lecture notes. The function $f$ is here represented using Numpy's polynomial subpackage. There are two types of tests, one that samples random polynomial coefficients, and one that samples random polynomial roots. Since we are only looking for real solutions, polynomials with only complex roots are not used as test cases.

In [5]:
def newtons_method(f):
  x = 0
  df = f.deriv(1)
  while abs(f(x)) > 0.000000001:
    x = x - f(x)/df(x)
  return x

class NewtonTest(unittest.TestCase):
  def test_random_coef(self):
    for n in range(2, 10):
      real = False
      while not real:
        y = np.random.rand(n)
        f = P(y)
        for root in f.roots():
          if root.imag == 0:
            real = True
      with self.subTest(f=f):
        x = newtons_method(f)
        np.testing.assert_almost_equal(f(x), 0)

  def test_random_solution(self):
    for n in range(1, 10):
      y = np.random.rand(n)
      # (y1-x)(y2-x)...
      f = reduce(lambda f, yi: f * P([-yi,1]), y, P([1]))
      x = newtons_method(f)
      with self.subTest(y=y, x=x, f=f):
        np.testing.assert_almost_equal(f(x), 0)
        self.assertTrue(np.isclose(x-y, 0, atol=1e-3).any())

# **Results**

All the above defined test cases pass.

In [6]:
unittest.main(argv=[''], verbosity=1, exit=False)

....
----------------------------------------------------------------------
Ran 4 tests in 0.820s

OK


# **Discussion**

All the implemented methods are valid ways of solving different types of equations. Naively sampling problem instances works as a way of testing the solver, but as the dimensionality grows the likelihood of sampling a solvable problem instance can decrease. If possible, it may be faster to sample a random solution, and derive the problem instance instead.